In [9]:
from finance.mongo.extraction import query_mongodb
from finance.params import *
import numpy as np
import pandas as pd


In [10]:
exchange_ls = ['NASDAQ']
table_name = 'company_profile'


In [11]:
result = query_mongodb(exchange_ls=exchange_ls, table=table_name)

exchange {} - 12568


In [12]:
result[0]['description'] == None

True

In [13]:
result_df = pd.DataFrame(result)
result_df

,_id,symbol,price,beta,volAvg,mktCap,lastDiv,range,changes,companyName,...,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund
0,657dc9f9b2cffe5b07e21e8e,SSSSL,23.3966,0.000000,1214,1.070344e+08,1.5,22.28-25.0,0.1516,SuRo Capital Corp. 6.00% Notes due 2026,...,None,NaN,0.000000,https://financialmodelingprep.com/image-stock/...,2021-12-17,True,False,True,False,False
1,657dc9f9b2cffe5b07e21e8f,LVAC,10.4300,0.000000,44325,1.499312e+08,0.0,9.94-10.85,0.0000,LAVA Medtech Acquisition Corp.,...,02451,NaN,0.000000,https://financialmodelingprep.com/image-stock/...,2021-12-17,False,False,False,False,False
2,657dc9fab2cffe5b07e21e90,CPLP,13.8000,0.904000,38893,2.753293e+08,0.6,11.93-16.1,0.0500,Capital Product Partners L.P.,...,18537,-124.487700,138.809204,https://financialmodelingprep.com/image-stock/...,2007-03-30,True,False,True,False,False
3,657dc9fdb2cffe5b07e21e96,CMTL,7.9200,1.474000,276725,2.255537e+08,0.4,7.26-16.87,0.0600,Comtech Telecommunications Corp.,...,11747,2.188480,12.671500,https://financialmodelingprep.com/image-stock/...,1980-03-17,False,False,True,False,False
4,657dc9feb2cffe5b07e21e98,VLON,0.4000,-1.148559,8091680,5.392920e+06,0.0,0.2054-16.5,-0.0600,"Vallon Pharmaceuticals, Inc.",...,19103,NaN,0.000000,https://financialmodelingprep.com/image-stock/...,2021-02-10,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12563,657e3188b2cffe5b07e2f59c,TACT,6.9900,1.937000,20630,6.960726e+07,0.0,4.92-9.44,-0.0200,TransAct Technologies Incorporated,...,06518,1.215880,8.755880,https://financialmodelingprep.com/image-stock/...,1996-08-22,False,False,True,False,False
12564,657e318ab2cffe5b07e2f5a0,KVSA,10.7400,0.072000,8044,7.431499e+07,0.0,9.94-11.22,0.0067,Khosla Ventures Acquisition Co.,...,94025,NaN,0.000000,https://financialmodelingprep.com/image-stock/...,2021-03-04,False,False,False,False,False
12565,657e3192b2cffe5b07e2f5b2,TAYD,24.0800,0.901000,6730,8.479483e+07,0.0,10.5-26.4,0.0281,"Taylor Devices, Inc.",...,14120,-8.348660,32.428665,https://financialmodelingprep.com/image-stock/...,1980-11-05,False,False,True,False,False
12566,657e3194b2cffe5b07e2f5b7,RNERW,0.2800,0.000000,0,0.000000e+00,0.0,0.1705-0.2999,0.0046,Mount Rainier Acquisition Corp.,...,10018,NaN,0.000000,https://financialmodelingprep.com/image-stock/...,2022-01-03,False,False,False,False,False


In [14]:
result_df.dropna(subset='description', inplace=True)
result_df.dropna(subset=['sector', 'industry'], inplace=True)
result_df.drop_duplicates(subset='symbol', inplace=True)

In [15]:
result_df.shape

(5872, 37)

In [16]:
result_description = result_df.to_dict('records')

In [26]:
import chromadb
from chromadb.utils import embedding_functions

chroma_client = chromadb.Client()
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

In [29]:
chroma_client.delete_collection('general_profile_mililm_l6_v2')

In [30]:
collection = chroma_client.create_collection(name="general_profile_mililm_l6_v2", embedding_function=sentence_transformer_ef)

In [45]:
documents = [i['description'] for i in result_description]
ids = [i['symbol'] for i in result_description]
metadatas = [{'symbol':i['symbol'], 'exchangeShortName':i['exchangeShortName'], \
            'industry':i['industry'], 'sector':i['sector'], 'companyName':i['companyName']}\
        for i in result_description]

In [46]:
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids)

Insert of existing embedding ID: LVAC
Insert of existing embedding ID: CPLP
Insert of existing embedding ID: CMTL
Insert of existing embedding ID: VLON
Insert of existing embedding ID: KLAC
Insert of existing embedding ID: LEE
Insert of existing embedding ID: PNT
Insert of existing embedding ID: APM
Insert of existing embedding ID: DAOOU
Insert of existing embedding ID: RVPH
Insert of existing embedding ID: OBCI
Insert of existing embedding ID: PEGRU
Insert of existing embedding ID: ALPP
Insert of existing embedding ID: TWCTU
Insert of existing embedding ID: LSCC
Insert of existing embedding ID: IDN
Insert of existing embedding ID: EXPO
Insert of existing embedding ID: FFBC
Insert of existing embedding ID: CZR
Insert of existing embedding ID: MCAD
Insert of existing embedding ID: BLCM
Insert of existing embedding ID: NOAC
Insert of existing embedding ID: FNVTW
Insert of existing embedding ID: ENERU
Insert of existing embedding ID: CENQ
Insert of existing embedding ID: LION
Insert of ex

In [48]:
chroma_client.get_collection(name="general_profile_mililm_l6_v2").peek() 

{'ids': ['AACG',
  'AACI',
  'AACIU',
  'AACQU',
  'AADI',
  'AAL',
  'AAME',
  'AAOI',
  'AAON',
  'AAPL'],
 'embeddings': [[-0.022555116564035416,
   -0.10829586535692215,
   -0.014604901894927025,
   -0.008347716182470322,
   0.022736744955182076,
   0.018328027799725533,
   -0.009448245167732239,
   -0.07032938301563263,
   -0.028704969212412834,
   -0.0025810967199504375,
   0.0606011264026165,
   0.08491083979606628,
   0.07809174805879593,
   -0.001166151836514473,
   -0.017305543646216393,
   -0.009861761704087257,
   0.03384624049067497,
   -0.12490827590227127,
   0.04768252372741699,
   -0.10290983319282532,
   -0.013648338615894318,
   -0.02201780676841736,
   0.005123850889503956,
   0.005074203480035067,
   -0.023549338802695274,
   -0.026973005384206772,
   0.06879148632287979,
   -0.028457099571824074,
   0.01083450485020876,
   -0.02065911516547203,
   0.025220168754458427,
   0.03999921679496765,
   -0.02293473295867443,
   0.02293609455227852,
   0.03841535747051239,

In [49]:
results = collection.query(
    query_texts=["companies crisping and gene editing"],
    n_results=10
)
results

{'ids': [['CRSP',
   'EDIT',
   'TWST',
   'LOGC',
   'BIOTW',
   'VERV',
   'ADVM',
   'TBIO',
   'DNAY',
   'BNTC']],
 'distances': [[0.7977274060249329,
   0.8159670233726501,
   0.9065848588943481,
   0.9164955019950867,
   0.978967010974884,
   0.9989222288131714,
   1.0402755737304688,
   1.0595464706420898,
   1.0595465898513794,
   1.0620496273040771]],
 'metadatas': [[{'exchangeShortName': 'NASDAQ',
    'industry': 'Biotechnology',
    'sector': 'Healthcare',
    'symbol': 'CRSP'},
   {'exchangeShortName': 'NASDAQ',
    'industry': 'Biotechnology',
    'sector': 'Healthcare',
    'symbol': 'EDIT'},
   {'exchangeShortName': 'NASDAQ',
    'industry': 'Diagnostics & Research',
    'sector': 'Healthcare',
    'symbol': 'TWST'},
   {'exchangeShortName': 'NASDAQ',
    'industry': 'Biotechnology',
    'sector': 'Healthcare',
    'symbol': 'LOGC'},
   {'exchangeShortName': 'NASDAQ',
    'industry': 'Shell Companies',
    'sector': 'Financial Services',
    'symbol': 'BIOTW'},
   {'exc

In [50]:
results['metadatas'][0]

[{'exchangeShortName': 'NASDAQ',
  'industry': 'Biotechnology',
  'sector': 'Healthcare',
  'symbol': 'CRSP'},
 {'exchangeShortName': 'NASDAQ',
  'industry': 'Biotechnology',
  'sector': 'Healthcare',
  'symbol': 'EDIT'},
 {'exchangeShortName': 'NASDAQ',
  'industry': 'Diagnostics & Research',
  'sector': 'Healthcare',
  'symbol': 'TWST'},
 {'exchangeShortName': 'NASDAQ',
  'industry': 'Biotechnology',
  'sector': 'Healthcare',
  'symbol': 'LOGC'},
 {'exchangeShortName': 'NASDAQ',
  'industry': 'Shell Companies',
  'sector': 'Financial Services',
  'symbol': 'BIOTW'},
 {'exchangeShortName': 'NASDAQ',
  'industry': 'Biotechnology',
  'sector': 'Healthcare',
  'symbol': 'VERV'},
 {'exchangeShortName': 'NASDAQ',
  'industry': 'Biotechnology',
  'sector': 'Healthcare',
  'symbol': 'ADVM'},
 {'exchangeShortName': 'NASDAQ',
  'industry': 'Medical Devices',
  'sector': 'Healthcare',
  'symbol': 'TBIO'},
 {'exchangeShortName': 'NASDAQ',
  'industry': 'Medical Devices',
  'sector': 'Healthcare'

In [52]:
results['documents']

[["CRISPR Therapeutics AG, a gene editing company, focuses on developing gene-based medicines for serious diseases using its proprietary Clustered Regularly Interspaced Short Palindromic Repeats (CRISPR)/CRISPR-associated protein 9 (Cas9) platform. Its CRISPR/Cas9 is a gene editing technology that allows for precise directed changes to genomic DNA. The company has a portfolio of therapeutic programs across a range of disease areas, including hemoglobinopathies, oncology, regenerative medicine, and rare diseases. The company's lead product candidate is CTX001, an ex vivo CRISPR gene-edited therapy for treating patients suffering from transfusion-dependent beta-thalassemia or severe sickle cell disease in which a patient's hematopoietic stem cells are engineered to produce high levels of fetal hemoglobin in red blood cells. It also develops CTX110, a donor-derived gene-edited allogeneic CAR-T investigational therapy targeting cluster of differentiation 19 positive malignancies; CTX120, a